<a href="https://colab.research.google.com/github/swatikdma/Sentiment-Analysis-Over-Streaming-Data/blob/main/SentimentAnalysis_logisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*  Input file  (for classification): /content/drive/MyDrive/Sentiment_Analysis/training.1600000.processed.noemoticon.csv










## Import modules and create spark session

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Supervised Learning

## Spark Installation On Colab

In [3]:
!rm -rf /content/spark-3.1.1-bin-hadoop2.7
!rm /content/spark-3.1.1-bin-hadoop2.7.tgz

rm: cannot remove '/content/spark-3.1.1-bin-hadoop2.7.tgz': No such file or directory


In [4]:
# -*- coding: utf-8 -*-
"""Spark_Installation_on_Colab.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/10n_cNyB482MZtKnxrPjl7zBwv8T3nP6s

# Colab is a Linux environment. 

# You can execute Linux commands from Colab notebook with a "!" prefix

## Install JDK
"""

# !apt-get install openjdk-8-jdk-headless -qq > /dev/null

"""## Get Spark installer (Check the path on Spark.org)"""

!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz

"""## Check if the file is copied"""

!ls

"""## Untar the Spark installer"""

!tar -xvf spark-3.1.1-bin-hadoop2.7.tgz

"""## Check the spark folder after untar"""

!ls




drive  sample_data  spark-3.1.1-bin-hadoop2.7.tgz
spark-3.1.1-bin-hadoop2.7/
spark-3.1.1-bin-hadoop2.7/NOTICE
spark-3.1.1-bin-hadoop2.7/kubernetes/
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/python_executable_check.py
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/autoscale.py
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/worker_memory_check.py
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/py_container_checks.py
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/decommissioning.py
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/pyfiles.py
spark-3.1.1-bin-hadoop2.7/kubernetes/tests/decommissioning_cleanup.py
spark-3.1.1-bin-hadoop2.7/kubernetes/dockerfiles/
spark-3.1.1-bin-hadoop2.7/kubernetes/dockerfiles/spark/
spark-3.1.1-bin-hadoop2.7/kubernetes/dockerfiles/spark/decom.sh
spark-3.1.1-bin-hadoop2.7/kubernetes/dockerfiles/spark/entrypoint.sh
spark-3.1.1-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/
spark-3.1.1-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindi

In [5]:
"""## Install findspark - a Python library to find Spark"""

!pip install -q findspark

"""## Set environment variables
Set Java and Spark Home based on location where they are stored
"""

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7" 

"""## Create a local Spark Session"""
import findspark
findspark.init()
from pyspark.sql import SparkSession

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

#spark = SparkSession.builder.master("local[*]").getOrCreate()

"""## Test Installation"""

df = spark.createDataFrame([{"Google": "Colab","Spark": "Scala"} ,{"Google": "Dataproc","Spark":"Python"}])
df.show()


+--------+------+
|  Google| Spark|
+--------+------+
|   Colab| Scala|
|Dataproc|Python|
+--------+------+



In [6]:
spark

In [7]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-05-21 08:55:08--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.223.73.198, 54.163.62.142, 3.208.247.60, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.223.73.198|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  37.3MB/s    in 0.4s    

2021-05-21 08:55:08 (37.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://34cc324bb9a8.ngrok.io


## Importing modules

In [8]:
#import modules
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover

## Read data file into Spark dataFrame

##Giving the title row to the data and finally creating a dataframe

In [25]:

PATH = "/content/drive/MyDrive/Sentiment_Analysis/training.1600000.processed.noemoticon.csv"

from pyspark.sql.types import *
from pyspark.sql.functions import *
schema = StructType([\
    StructField("Target", IntegerType(), True),\
    StructField("Id", LongType(), True),\
    StructField("Date", StringType(), True),\
    StructField("Flag", StringType(), True),\
    StructField("User", StringType(), True),\
    StructField("Tweet", StringType(), True)])

tweets_csv = spark.read.format("csv")\
.option("header", "false")\
.schema(schema)\
.load(PATH)

tweets_csv.show()


+------+----------+--------------------+--------+---------------+--------------------+
|Target|        Id|                Date|    Flag|           User|               Tweet|
+------+----------+--------------------+--------+---------------+--------------------+
|     0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|     0|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|     0|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|     0|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|     0|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|     0|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|     0|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|     0|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|     0|1467811795|Mon Apr 06 22:20:...|NO_

In [10]:
tweets_csv.printSchema()

root
 |-- Target: integer (nullable = true)
 |-- Id: long (nullable = true)
 |-- Date: string (nullable = true)
 |-- Flag: string (nullable = true)
 |-- User: string (nullable = true)
 |-- Tweet: string (nullable = true)



## Select the related data

In [26]:
#select only "SentimentText" and "Sentiment" column, 
#and cast "Sentiment" column data into integer
data = tweets_csv.select("Tweet", col("Target").cast("Int").alias("label"))
data.show(truncate = False,n=5)

+-------------------------------------------------------------------------------------------------------------------+-----+
|Tweet                                                                                                              |label|
+-------------------------------------------------------------------------------------------------------------------+-----+
|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D|0    |
|is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!    |0    |
|@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds                          |0    |
|my whole body feels itchy and like its on fire                                                                     |0    |
|@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.     |0    |
+-------

In [36]:
import pyspark.sql.functions as f
my_list = data.select(f.collect_list('label')).first()[0]
my_list = set(my_list)
my_list

{0, 4}

## Divide data into training and testing data

In [12]:
#divide data, 70% for training, 30% for testing
dividedData = data.randomSplit([0.75, 0.25]) 
trainingData = dividedData[0] #index 0 = data training
testingData = dividedData[1] #index 1 = data testing
train_rows = trainingData.count()
test_rows = testingData.count()
print ("Training data rows:", train_rows, "; Testing data rows:", test_rows)

Training data rows: 1200076 ; Testing data rows: 399924


## Prepare training data

Separate "SentimentText" into individual words using tokenizer

In [13]:
from pyspark.ml.feature import StopWordsRemover, RegexTokenizer

In [14]:
tokenizer = RegexTokenizer(pattern=r'(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', inputCol='Tweet', outputCol='out')
tokenizedTrain = tokenizer.transform(trainingData)
tokenizedTrain.show(truncate=False,n=500)


+---------------------------------------------------------------------------------------------------------------------------------------------------+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|Tweet                                                                                                                                              |label|out                                                                                                                                                      |
+---------------------------------------------------------------------------------------------------------------------------------------------------+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                           exhausted                 

Removing stop words (unimportant words to be features)

In [15]:
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                       outputCol="MeaningfulWords")
SwRemovedTrain = swr.transform(tokenizedTrain)
SwRemovedTrain.show( n=100)

+--------------------+-----+--------------------+--------------------+
|               Tweet|label|                 out|     MeaningfulWords|
+--------------------+-----+--------------------+--------------------+
|                 ...|    0|[                ...|[                ...|
|                 ...|    0|[                ...|[                ...|
|                 ...|    0|[                ...|[                ...|
|               I ...|    0|[               i...|[               i...|
|               ju...|    0|[               j...|[               j...|
|            Miss ...|    0|[            miss...|[            miss...|
|          .. Omga...|    0|[          ,  omg...|[          ,  omg...|
|         or i jus...|    0|[         or i ju...|[         or i ju...|
|        wish ella...|    0|[        wish ell...|[        wish ell...|
|       FS keeps c...|    0|[       fs keeps ...|[       fs keeps ...|
|       i really2 ...|    0|[       i really2...|[       i really2...|
|     

Converting words feature into numerical feature. In Spark 2.2.1,it is implemented in HashingTF funtion using Austin Appleby's MurmurHash 3 algorithm

In [16]:
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
numericTrainData = hashTF.transform(SwRemovedTrain).select(
    'label', 'MeaningfulWords', 'features')
numericTrainData.show(truncate=False, n=100)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------+
|label|MeaningfulWords                                                                                                                              |features                                                                                         |
+-----+---------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------+
|0    |[                                           exhausted]                                                                                       |(262144,[208793],[1.0])                                                                          |
|0    |[

## Prepare testing data

In [17]:
from pyspark.ml.feature import StopWordsRemover, RegexTokenizer

In [18]:
tokenizer = RegexTokenizer(pattern=r'(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', inputCol='Tweet', outputCol='out')
tokenizedTest = tokenizer.transform(testingData)
#tokenizedTest.show(n=500)


In [19]:
#tokenizedTest = tokenizer.transform(testingData)
SwRemovedTest = swr.transform(tokenizedTest)
numericTest = hashTF.transform(SwRemovedTest).select(
    'Label', 'MeaningfulWords', 'features')
numericTest.show(truncate=False, n=2)


+-----+-----------------------------------------------------------------------------+--------------------------------------------+
|Label|MeaningfulWords                                                              |features                                    |
+-----+-----------------------------------------------------------------------------+--------------------------------------------+
|0    |[                                     i miss her so much already]            |(262144,[249707],[1.0])                     |
|0    |[             i just want to hear from you,  i guess that, s asking too much]|(262144,[51347,129629,225418],[1.0,1.0,1.0])|
+-----+-----------------------------------------------------------------------------+--------------------------------------------+
only showing top 2 rows



## Train our classifier model using training data

In [ ]:
# lr = LogisticRegression(labelCol="label", featuresCol="features", 
#                         maxIter=10, regParam=0.01)
# model = lr.fit(numericTrainData)
# print ("Training is done!")

In [ ]:
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# instantiate the base classifier.
lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)

# instantiate the One Vs Rest Classifier.
ovr = OneVsRest(classifier=lr)

# train the multiclass model.
model = ovr.fit(numericTrainData)



In [39]:
# score the model on test data.
predictions = model.transform(numericTrainData)

# obtain evaluator.
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

# compute the classification error on test data.
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.244271


In [43]:
predictions

DataFrame[label: int, MeaningfulWords: array<string>, features: vector, rawPrediction: string, prediction: double]

In [52]:
import pyspark.sql.functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType

preds_and_labels = predictions.select(['prediction','label']).withColumn('label', F.col('label').cast(FloatType())).orderBy('prediction')
#select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','label'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

print(metrics.confusionMatrix().toArray())

[[453636. 146392.]
 [146752. 453296.]]


## Predict testing data and calculate the accuracy model

In [57]:
prediction = model.transform(numericTest)
predictionFinal = prediction.select(
    "MeaningfulWords", "prediction", "Label")
# predictionFinal.show(n=4, truncate = False)
correctPrediction = predictionFinal.filter(
    predictionFinal['prediction'] == predictionFinal['Label']).count()
totalData = predictionFinal.count()
print("correct prediction:", correctPrediction, ", total data:", totalData, ", accuracy:", correctPrediction/totalData)

correct prediction: 233364 , total data: 399924 , accuracy: 0.5835208689651034
